# Statistical test of Treatment A & B

This is the analysis of Treatment A & B for the novo case for presentation on 2022-03-28 @ 13.00



In [1]:
'''
This section will simply import the necessary modules to run the program. 
These are all standard Python modules, so nothing strange should happen here.
PLEASE have internet! 
'''

import sys

# Install modules, if not already installed
!{sys.executable} -m pip install numpy pandas matplotlib statistics

print('prereq modules installed')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

print('imported modules')

Defaulting to user installation because normal site-packages is not writeable
prereq modules installed
imported modules


# Clean up of data

This section entails the cleanup of data. 
To avoid data curruption, The cleaned up data is saved into two new sheet in in the file 'cleaned-data.xlsx'.

Initial data-inspection provided some thoughts.
As we need to provide a descriptive table, we have some small issues with the data.


For ADSL:
- Three subjects are missing DIABDUR. i.e. ADSL Line 16 (Subject S001047) Is missing entry at DIABDUR and DIABDURU, So we can expect a few other subjects to be missing some numbers
- S005025 is VERY tall. I believe it is a safe assumption that this subject was 181cm tall, and not 181 m tall, This should be corrected.
- Weight is noted in both LB and KG, however, the table specifies KG, so we should convert all lb to kg. 

For ADLB:
- Data is in Long format. Could be smart to correct it to wide, to ensure we are not missing any data for Visit 10, 14, 18, 22, 26, 30. 
    - This means that AVSIT should be segmented into Visitv2, and Week columns.
- We are dividing the data into two PARAMCD's (of interest here at least) (at a glance these two correspond to FPG and HbA1c.. I will have to check to make sure)
    - This might actually be smarter to seperate the data into two temporary dataframes here, so no confusion occurs here. And to ensure that no data is missing.
- There has also been some issues with a few of the acquisitions of data. see ANL01FL != 'Y'
    - see 'Retest Rule' and 'oroginal visit realloc'. Not sure what that means... 
        - In a real scenario I really should know this
- Lets also make sure that the units for FPG remains mmol/L and HbA1c remains %
- While we are checking, maybe some of the PARAM's are not only FPG or HbA1c.




In [2]:
# Specify variable constants

MASTER_XLSX_NAME = 'src/data.xlsx'
CLEAN_XLSX_NAME = 'src/clean-data.xlsx'


In [53]:
def convertHeightCorrectly(value:float, unit:str) -> float:
    """
    This function converts height from different unit (etc cm)
    to m.
    Also ensures that if height is < 100
    """
    
    unit_conv = float(value<100) + float(value>=100) * 0.01
    if unit.lower() != 'm':
        return np.nan

    return value * unit_conv

    
def converWgt2Kg(value:float, unit:str) -> float:
    """
    Converts weight into Kg
    - if lb -> wgt * fLB2KG
    else return wgt
    """
    fLB2KG = 0.4535924 # conversion ratio from lb to kg
    unit_conv = float(unit == 'lb') * fLB2KG + float(unit == 'kg')
    return value * unit_conv

def convertMonthsToYears(value:float, unit:str) -> float:
    """
    Convert months into years
    """
    
    unit_conv = float(unit=='years' or unit=='years'.upper()) + float(unit=='months')*1.0/12.0
    return value * unit_conv

def my_filter(dfl, dfr):
    """
    Filter system
    """
    
    return 



In [66]:
# Open MASTER dataframe
MASTER_DATAFRAME = pd.read_excel(MASTER_XLSX_NAME, sheet_name=None)

clean_df = pd.DataFrame()

for sheetname, df_sheet in MASTER_DATAFRAME.items():
    # Iterate through every sheet in the master dataframe
    print(sheetname)
    if sheetname == 'ADSL':
        # (sanity check), make sure that no subject is written twice here.
        ADSL_num_dub_subj = len(df_sheet[df_sheet.SUBJID.duplicated()])
        ADSL = df_sheet.copy()
        # Create body weight DF
        clean_df['AgeYears']              = df_sheet.apply(lambda row: convertMonthsToYears(  value=row.AGE    , unit=row.AGEU),     axis=1)
        clean_df['HeightM']               = df_sheet.apply(lambda row: convertHeightCorrectly(value=row.HGTBL  , unit=row.HGTBLU),   axis=1)
        clean_df['BodyWeightKg']          = df_sheet.apply(lambda row: converWgt2Kg(          value=row.WGTBL  , unit=row.WGTBLU),   axis=1)
        clean_df['DiabetesDurationYears'] = df_sheet.apply(lambda row: convertMonthsToYears(  value=row.DIABDUR, unit=row.DIABDURU), axis=1)
        #print(clean_ADSL_df)
        #print(f'{sheetname} > Number of dublicate subj entries (should be 0) : {ADSL_num_dub_subj}')
        continue
    
    if sheetname == 'ADLB':
        ADLB = df_sheet.copy()
        
        # Check 
        continue


print(clean_df.describe())


ADSL
ADLB
         AgeYears     HeightM  BodyWeightKg  DiabetesDurationYears
count  257.000000  257.000000    257.000000             254.000000
mean    59.634241    1.686701     89.261828              12.405512
std      9.094903    0.099893     16.840872               6.403813
min     29.000000    1.460000     53.523903               2.000000
25%     54.000000    1.600200     78.471485               8.000000
50%     61.000000    1.690000     87.996920              11.000000
75%     66.000000    1.752600     99.600000              16.000000
max     83.000000    1.930400    147.417530              36.000000


In [106]:
tmpdf = clean_df.copy()
tmpdf['PARAMCD']='C64849B'
tmpdf['VISITNUM']=10
selectADLB = ADLB.loc[:, ['PARAMCD', 'VISITNUM', 'FASFL']]

tf1 = ADLB[selectADLB.isin(['C64849B', 10, 'Y']).all(axis=1)]
print(tf1[tf1.SUBJID.duplicated()])

# selecttmpdf = tmpdf.loc[:, ['SUBJID', 'PARAMCD', 'VISITNUM']]

       SUBJID FASFL         TRTP             AVISIT  VISITNUM  \
2127  S101066     Y  Treatment B  Visit 10 (Week 0)        10   

                PARAM  PARAMCD  AVAL AVALU ANL01FL     ANL01REA  
2127  HbA1c Blood (%)  C64849B   7.9     %       N  Retest rule  
